In [51]:
from common.inputreader import InputReader, PuzzleWrapper
from common.matrix import Matrix, MatrixNavigator

puzzle = PuzzleWrapper(year=2024, day=int("18"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# RAM Run

[Open Website](https://adventofcode.com/2024/day/18)

In [52]:
# helper functions
def domain_from_input(input: InputReader, size: int) -> (Matrix, list):
    lines = input.lines_as_strs()

    # find points
    points = []
    for line in lines:
        point = line[0].split(",")
        points.append((int(point[0]), int(point[1])))

    # create matrix
    grid = []
    for y in range(size + 1):
        row = []
        for x in range(size + 1):
            row.append(".")
        grid.append(row)

    return Matrix(grid), points


test_input, queue = domain_from_input(puzzle.example(0), 6)
test_input.print()
print(queue)

.......
.......
.......
.......
.......
.......
.......
[(5, 4), (4, 2), (4, 5), (3, 0), (2, 1), (6, 3), (2, 4), (1, 5), (0, 6), (3, 3), (2, 6), (5, 1), (1, 2), (5, 5), (2, 5), (6, 5), (1, 4), (0, 4), (6, 4), (1, 1), (6, 1), (1, 0), (0, 5), (1, 6), (2, 0)]


In [53]:
from common.matrix import Direction, MatrixNavigator

directions = [Direction.UP, Direction.DOWN, Direction.LEFT, Direction.RIGHT]


def find_shortest_path(matrix: Matrix, start: (int, int), end: (int, int)) -> int:
    queue = [(start, Direction.DOWN, 0), (start, Direction.RIGHT, 0)]
    history = []

    while queue:
        # short queue by cost
        for i in range(1, len(queue)):
            if queue[i][2] < queue[0][2]:
                queue[0], queue[i] = queue[i], queue[0]

        # pop the node
        point, direction, cost = queue.pop(0)

        # check if we are done
        if point == end:
            return cost

        # if point and direction in history stop
        if (point, direction) in history:
            continue
        history.append((point, direction))

        # move the pointer
        pointer = MatrixNavigator(matrix, point[0], point[1])
        pointer.move(direction)

        # check all directions
        for new_direction in directions:
            ok, value = pointer.peek_value(new_direction)
            if ok and value == ".":
                queue.append((pointer.get_position(), new_direction, cost + 1))


# test case (part 1)
def part_1(reader: InputReader, size: int, steps: int, debug: bool) -> int:
    matrix, queue = domain_from_input(reader, size)

    for i in range(0, steps):
        point = queue[i]
        matrix.set_value(point[0], point[1], "#")

    if debug:
        matrix.print()

    start = (0, 0)
    end = (size, size)
    return find_shortest_path(matrix, start, end)


result = part_1(puzzle.example(0), 6, 12, True)
display(result)
assert result == 22

...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....


22

In [54]:
# real case (part 1)
result = part_1(puzzle.input(), 70, 1024, False)
display(result)
assert result == 304

304

In [55]:
def find_shortest_path_2(matrix: Matrix, start: (int, int), end: (int, int)) -> list:
    queue = [(start, Direction.DOWN, 0, []), (start, Direction.RIGHT, 0, [])]
    known = []

    while queue:
        # short queue by cost
        for i in range(1, len(queue)):
            if queue[i][2] < queue[0][2]:
                queue[0], queue[i] = queue[i], queue[0]

        # pop the node
        point, direction, cost, history = queue.pop(0)

        # check if we are done
        if point == end:
            return history

        # if point and direction in history stop
        if (point, direction) in known:
            continue
        known.append((point, direction))

        # move the pointer
        pointer = MatrixNavigator(matrix, point[0], point[1])
        pointer.move(direction)

        # check all directions
        for new_direction in directions:
            ok, value = pointer.peek_value(new_direction)
            if ok and value == ".":
                history.append((point))
                queue.append((pointer.get_position(), new_direction, cost + 1, history))

    return []


# test case (part 2)
def part_2(reader: InputReader, size: int, skip: int) -> str:
    matrix, queue = domain_from_input(reader, size)

    last_steps = None
    count = 0
    start = (0, 0)
    end = (size, size)

    while True:
        count += 1
        point = queue.pop(0)
        matrix.set_value(point[0], point[1], "#")
        if skip > 0:
            skip -= 1
            continue

        # if this is the first time we are here, run the shortest path
        if last_steps is None:
            last_steps = find_shortest_path_2(matrix, start, end)

        # if point in last steps run simulation
        if point in last_steps:
            print(f"start simulation as {count}")
            last_steps = find_shortest_path_2(matrix, start, end)
            print("done")

        # if we are done
        if len(last_steps) == 0:
            return f"{point[0]},{point[1]}"


result = part_2(puzzle.example(0), 6, 5)
display(result)
assert result == "6,1"

start simulation as 7
done
start simulation as 8
done
start simulation as 9
done
start simulation as 10
done
start simulation as 11
done
start simulation as 12
done
start simulation as 13
done
start simulation as 14
done
start simulation as 15
done
start simulation as 17
done
start simulation as 18
done
start simulation as 20
done
start simulation as 21
done


'6,1'

In [56]:
# real case (part 2)
result = part_2(puzzle.input(), 70, 2800)
display(result)

start simulation as 2805
done
start simulation as 2851
done
start simulation as 2864
done
start simulation as 2876
done
start simulation as 2877
done


'50,28'

In [57]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="Pun intended.">Run</span> (Pun intended.)